In [7]:
from selenium.webdriver.support.ui import Select
import time

class HREF_SCRETCHER_DABANG(object):
    
    def __init__(self, dong_lst):
        dong_lst = [i.strip() for i in dong_lst] 
        self.dong_lst = dong_lst
    
    
    def checkout_duplicated(self,fname):
        df = pd.read_csv(fname)
        url = 'https://www.dabangapp.com/room/'
        copy = url + df['id'] 
        copy = set(copy)
        dong_lst = set(self.dong_lst)
        return dong_lst - copy
    
    
    def extract_href(self):
        href_lst = []
        chromedriver = './chromedriver'
        driver = webdriver.Chrome(chromedriver)
        driver.get('https://www.dabangapp.com/search#/map?type=region&id=11140162&position=%7B%22center%22%3A%5B127.01550714548125%2C37.55690177250604%5D%2C%22zoom%22%3A14%7D&filters=%7B%22deposit-range%22%3A%5B0%2C999999%5D%2C%22price-range%22%3A%5B0%2C999999%5D%2C%22room-type%22%3A%5B0%2C1%2C2%2C3%2C4%2C5%5D%2C%22deal-type%22%3A%5B0%2C1%5D%2C%22location%22%3A%5B%5B126.97662584360137%2C37.537813508505565%5D%2C%5B127.05438844736113%2C37.57598514804419%5D%5D%7D&cluster=%7B%22name%22%3A%22%EC%8B%A0%EB%8B%B9%EB%8F%99%22%7D')
        for i in self.dong_lst:
            text_box = driver.find_element_by_xpath("//input[@class='SearchForm-input form-control']")
            text_box.clear()
            address =  i
            address = address.decode('utf8')
            text_box.send_keys(address)
            time.sleep(5)
            try:
                click_box = driver.find_element_by_xpath("//ul[@class='SearchForm-list search-items']/li/span[@class='SearchForm-item name full']")
                text = click_box.text
            except:
                print 'text_error' 
            if text == address:    
                click_box.click() 
            else:
                print 'not match'
                continue    
            time.sleep(3)
            try:
                region_box = driver.find_elements_by_class_name("Room-item")
                for i,room_info in enumerate(region_box):
                    try:
                        link = room_info.find_element_by_tag_name('a')
                        href = link.get_attribute('href')
                        href_lst.append(href)
                    except:
                        print 'no a tag with href'
                
                        
                while True:
                    try:
                        time.sleep(3)
                        next_icon = driver.find_element_by_xpath("//ul[@class='Pagination']//a[@class='Pagination-item Pagination-item--next']")
                        next_icon.click()
                        region_box = driver.find_elements_by_class_name("Room-item")

                        for i,room_info in enumerate(region_box):
                            try:
                                link = room_info.find_element_by_tag_name('a')
                                href = link.get_attribute('href')
                                href_lst.append(href)
                            except:
                                print 'no a tag with href'
                                continue
                        
                    except: 
                        print 'no next icon'
                        break

                    
            
            except:
                print 'Nothing to do'
        driver.quit()
        return href_lst
    def findout_non_finded(self):
        
        chromedriver = './chromedriver'
        driver = webdriver.Chrome(chromedriver)
        driver.get('https://www.dabangapp.com/search#/map?type=region&id=11140162&position=%7B%22center%22%3A%5B127.01550714548125%2C37.55690177250604%5D%2C%22zoom%22%3A14%7D&filters=%7B%22deposit-range%22%3A%5B0%2C999999%5D%2C%22price-range%22%3A%5B0%2C999999%5D%2C%22room-type%22%3A%5B0%2C1%2C2%2C3%2C4%2C5%5D%2C%22deal-type%22%3A%5B0%2C1%5D%2C%22location%22%3A%5B%5B126.97662584360137%2C37.537813508505565%5D%2C%5B127.05438844736113%2C37.57598514804419%5D%5D%7D&cluster=%7B%22name%22%3A%22%EC%8B%A0%EB%8B%B9%EB%8F%99%22%7D')
        for i in self.dong_lst:
            text_box = driver.find_element_by_xpath("//input[@class='SearchForm-input form-control']")
            text_box.clear()
            address =  i
            address = address.decode('utf8')
            text_box.send_keys(address)
            time.sleep(5)
            try:
                click_box = driver.find_element_by_xpath("//ul[@class='SearchForm-list search-items']/li/span[@class='SearchForm-item name full']")
                text = click_box.text
            except:
                print 'text_error' 
            if text == address:    
                click_box.click() 
            else:
                print 'not match', address
                continue    
    
    def extract_final_lst(self):
        href_lst = self.extract_href()
        href_dict = Counter(href_lst)
        final_lst = href_dict.keys()
        self.final_lst = final_lst
        return final_lst
    
    def write_file(self, file_name):
        with open(file_name,'w') as txtfile:
            for i in self.final_lst:
                url = i + ' '
                txtfile.write(url)

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re
import unicodecsv as csv

class Href_to_data(object):
    def __init__(self,href_lst):
        self.href_lst = href_lst
    
    
    def href_to_csv(self,fname):
        with open(fname, 'w') as fw:
            writer = csv.writer(fw)
            writer.writerow(['id','loc','room_type','size', 'room_floor','building_floor','parking','elevator'
                         ,'heat_type','animal','aircondition','laundry','bed','desk','closet','tv'
                         ,'shoes','refrigerator','gas','induction','microwave','doar_lock','bidet'
                        ,'price'])
            for href in self.href_lst:
                web = requests.get(href)
                x = re.search("dabang.web.detail\((.*}})",web.content)
                jsonform = x.group(1)
                jsonf = json.loads(jsonform)
                loc = jsonf['room']['location']
                room_type =  jsonf['room']['room_type_str']
                size = jsonf['room']['room_size']
                room_floor = jsonf['room']['room_floor']
                building_floor = jsonf['room']['building_floor']
                elevator = jsonf['room']['elevator']
                parking = jsonf['room']['parking']
                heat_type = jsonf['room']['heating']
                animal = jsonf['room']['animal']
                options = jsonf['room']['room_options']
                room_id = jsonf['room']['id']
                aircondition = 0 in options
                laundry = 1 in options
                bed = 2 in options
                desk = 3 in options
                closet = 4 in options
                tv = 5 in options
                shoes = 6 in options
                refrigerator = 7 in options
                gas = 8 in options
                induction = 9 in options
                microwave = 10 in options
                doar_lock = 11 in options
                bidet = 12 in options
                if jsonf['room']['maintenance']:
                    mc = jsonf['room']['maintenance_cost']
                else: 
                    mc = 0
                if mc is None: 
                    mc = 0
                deposit = jsonf['room']['price_info'][0][0] 
                if deposit is None: 
                    deposit = 0
                rent = jsonf['room']['price_info'][0][1] 
                if rent is None :
                    rent = 0
                price = deposit + (rent + mc) * 100
                data_row = [room_id, loc, room_type, size, room_floor, building_floor, parking,
                                 elevator, heat_type, animal, aircondition, laundry, bed, desk,
                                 closet,tv, shoes, refrigerator, gas,induction, microwave,
                                 doar_lock,bidet,price]
                self.fname = fname
                writer.writerow(data_row)
                
    def csv_to_df(self):
        df = pd.read_csv(self.fname)
        return df


In [12]:
new_url = HREF_SCRETCHER_DABANG(['https://www.dabangapp.com/room/56dcf84d56943026cc32afd0'])

In [13]:
import pandas as pd
new_url.checkout_duplicated('updated_data.csv')

{'https://www.dabangapp.com/room/56dcf84d56943026cc32afd0'}